# 🩺 Classificação de Anomalias em ECG
## Fibrilação Atrial vs. Ritmo Sinusal Normal

**Projeto:** Detecção Automática de Arritmias Cardíacas  
**Bases de Dados:** PhysioNet MIT-BIH

---

### 📊 Objetivos do Projeto
1. Extrair características avançadas de sinais ECG
2. Comparar ritmo normal vs. fibrilação atrial
3. Construir modelo de classificação robusto
4. Avaliar poder discriminativo das features

---

## 🫀 Fundamentos: O que são Anomalias Cardíacas?

### Ritmo Sinusal Normal (NSR)
- Batimentos regulares e coordenados
- Ondas P bem definidas
- Intervalos R-R consistentes

### Fibrilação Atrial (AFib)
- **Ausência de ondas P**: Contração atrial descoordenada
- **Intervalos R-R irregulares**: Atividade elétrica caótica
- **Alta variabilidade**: Padrão imprevisível entre batimentos

![image.png](attachment:image.png)

### 💡 Hipótese Científica
A **Variabilidade da Frequência Cardíaca (HRV)** deve ser significativamente maior em pacientes com AFib devido à irregularidade característica dos intervalos R-R.

In [ ]:
# 📦 Importação de Bibliotecas
import wfdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from ecg_processor_V2 import process_records_in_windows, extract_comprehensive_features

# Bibliotecas para Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Configuração visual
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

---

## 📥 1. Extração de Dados

### Bases de Dados Utilizadas
- **MIT-BIH Normal Sinus Rhythm Database (nsrdb)**: 5 pacientes saudáveis
- **MIT-BIH Atrial Fibrillation Database (afdb)**: 5 pacientes com AFib

### Metodologia de Processamento
- **Janelas de 30 segundos**: Segmentação não-sobreposta
- **Lead I**: Derivação única para consistência
- **Múltiplas amostras por paciente**: Cada janela = 1 amostra

In [ ]:
# Configuração de diretórios
output_dir = 'data'
basic_csv_path = os.path.join(output_dir, 'ecg_basic_features_ECG2.csv')
comprehensive_csv_path = os.path.join(output_dir, 'ecg_comprehensive_features_ECG2.csv')

# Criar diretório se não existir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

---

## 🔬 2. Pipeline de Processamento de Sinais

### Etapas do Processamento

#### 1️⃣ **Filtragem Passa-Banda**
- Remove deriva da linha de base (0.5 Hz)
- Elimina ruído de alta frequência (40 Hz)
- Filtro Butterworth com fase zero

![image.png](attachment:image.png)

#### 2️⃣ **Detecção de Picos R**
- Identifica complexos QRS
- Base para segmentação de batimentos

![image-2.png](attachment:image-2.png)

#### 3️⃣ **Features de HRV (Domínio do Tempo)**
- `mean_rr`: Intervalo R-R médio
- `sdnn`: Desvio padrão dos intervalos NN
- `rmssd`: Raiz quadrada da média das diferenças sucessivas

#### 4️⃣ **PCA - Análise de Componentes Principais**
- Captura variações morfológicas dos batimentos
- Redução de dimensionalidade
- Identificação de padrões dominantes

![image-3.png](attachment:image-3.png)

#### 5️⃣ **ICA - Análise de Componentes Independentes**
- Separação cega de fontes
- Isola artefatos e sinais fisiológicos
- **Potencial para detectar ondas fibrilatórias**

#### 6️⃣ **Análise Espectral**
- Domínio da frequência (LF/HF)
- Balanço autonômico
- ⚠️ **Limitação**: Janelas de 30s são curtas para análise espectral completa

In [ ]:
print("🔄 Pipeline de Extração de Features ECG")
print("=" * 60)

reprocess = False  # Definir como True para reprocessar

if reprocess or not os.path.exists(basic_csv_path) or not os.path.exists(comprehensive_csv_path):
    # Lista de registros do PhysioNet
    nsr_records = ['16265', '16272', '16420', '16483', '16539']
    afib_records = ['04015', '04043', '04126', '04746', '04908']

    # 1. Extrair Features Básicas de HRV
    print("📊 Extraindo features básicas de HRV...")
    nsr_basic = process_records_in_windows(nsr_records, 'nsrdb/1.0.0/', 'Normal', use_comprehensive=False)
    afib_basic = process_records_in_windows(afib_records, 'afdb/1.0.0/', 'AFib', use_comprehensive=False)

    df_basic = pd.DataFrame(nsr_basic + afib_basic)
    df_basic.to_csv(basic_csv_path, index=False)
    print(f"✓ Features básicas: {len(df_basic)} janelas, {len(df_basic.columns)-3} features")

    # 2. Extrair Features Avançadas (PCA, ICA, Espectral)
    print(f"\n🧠 Extraindo features avançadas (PCA, ICA, Espectral)...")

    nsr_comprehensive = process_records_in_windows(nsr_records, 'nsrdb/1.0.0/', 'Normal', use_comprehensive=True)
    afib_comprehensive = process_records_in_windows(afib_records, 'afdb/1.0.0/', 'AFib', use_comprehensive=True)

    df_comprehensive = pd.DataFrame(nsr_comprehensive + afib_comprehensive)
    df_comprehensive.to_csv(comprehensive_csv_path, index=False)
    print(f"✓ Features avançadas: {len(df_comprehensive)} janelas, {len(df_comprehensive.columns)-3} features")

    # Resumo
    print(f"\n📈 RESUMO FINAL DE FEATURES")
    print("=" * 60)
    print(f"Features Básicas HRV: {len([col for col in df_basic.columns if col not in ['record', 'window_id', 'label']])}")
    print(f"Features Avançadas: {len([col for col in df_comprehensive.columns if col not in ['record', 'window_id', 'label']])}")
    print(f"Fator de Aumento: {len(df_comprehensive.columns) / len(df_basic.columns):.1f}x mais features")

    print(f"\n✅ Pronto para análise!")
else:
    print('✓ Arquivos CSV já existem!')
    print(f"✅ Pronto para análise!")

---

## 🧠 3. Por que PCA, ICA e Análise Espectral?

### 🎯 PCA (Análise de Componentes Principais)
**Objetivo:** Redução de dimensionalidade e extração de padrões morfológicos

- ✅ **Captura variações importantes** na morfologia dos batimentos
- ✅ **Redução de ruído**: Foca nas direções de maior variância
- ✅ **Reconhecimento de padrões**: Diferentes condições cardíacas = padrões morfológicos distintos
- ✅ **Compressão de dados**: Representa formas complexas com menos dimensões

### 🎯 ICA (Análise de Componentes Independentes)
**Objetivo:** Separação cega de fontes e remoção de artefatos

- ✅ **Separação de fontes mistas**: Isola sinais fisiológicos sobrepostos
- ✅ **Remoção de artefatos**: Separa ruído muscular, deriva de linha de base
- ✅ **Análise multi-processo**: Revela atividades cardíacas sobrepostas
- ✅ **Detecção de arritmias**: Diferentes arritmias = padrões independentes únicos

### 🎯 Análise Espectral
**Objetivo:** Análise no domínio da frequência

- ✅ **Domínio da frequência**: Analisa HRV em diferentes bandas
- ✅ **Função autonômica**: Razão LF/HF indica balanço simpático vs. parassimpático
- ✅ **Relevância clínica**: Marcadores estabelecidos para avaliação da saúde cardíaca

⚠️ **Nota sobre Limitações**: A razão LF/HF requer janelas de 5 minutos (padrão). Janelas de 30s são insuficientes para análise espectral robusta das bandas de baixa frequência.

In [ ]:
# Carregar dados processados
nsr_records = ['16265', '16272', '16420', '16483', '16539']
afib_records = ['04015', '04043', '04126', '04746', '04908']

df_basic = pd.read_csv(basic_csv_path)
df_comprehensive = pd.read_csv(comprehensive_csv_path)

print("✅ Dados carregados com sucesso!")
print(f"📊 Shape do dataset: {df_comprehensive.shape}")

---

## 📊 4. Análise Exploratória do Dataset

### Visão Geral dos Dados

In [ ]:
# Análise do Dataset
print("📊 ANÁLISE ABRANGENTE DO DATASET")
print("=" * 60)

# Informações básicas
print(f"Dimensões do Dataset: {df_comprehensive.shape}")
print(f"Janelas Normais: {len(df_comprehensive[df_comprehensive['label'] == 'Normal'])}")
print(f"Janelas AFib: {len(df_comprehensive[df_comprehensive['label'] == 'AFib'])}")

# Categorias de features
feature_categories = {
    'HRV (Domínio do Tempo)': [col for col in df_comprehensive.columns if any(hrv in col for hrv in ['mean_rr', 'sdnn', 'rmssd', 'pnn50', 'hr_mean'])],
    'PCA (Morfológicas)': [col for col in df_comprehensive.columns if 'pca_' in col],
    'ICA (Fontes Independentes)': [col for col in df_comprehensive.columns if 'ica_' in col],
    'Espectral (Domínio da Frequência)': [col for col in df_comprehensive.columns if 'spectral_' in col],
    'Qualidade do Sinal': [col for col in df_comprehensive.columns if any(qual in col for qual in ['signal_quality', 'num_heartbeats', 'beat_detection'])]
}

print(f"\n📂 Categorias de Features:")
for category, features in feature_categories.items():
    print(f"  • {category}: {len(features)} features")
    if len(features) > 0:
        example_features = features[:3] if len(features) >= 3 else features
        print(f"    Exemplos: {', '.join(example_features)}")

# Verificar valores faltantes
missing_counts = df_comprehensive.isnull().sum()
features_with_missing = missing_counts[missing_counts > 0]

if len(features_with_missing) > 0:
    print(f"\n⚠️ Features com valores faltantes:")
    for feature, count in features_with_missing.items():
        print(f"  • {feature}: {count} faltantes ({count/len(df_comprehensive)*100:.1f}%)")
else:
    print(f"\n✅ Nenhum valor faltante detectado!")

# Estatísticas básicas
print(f"\n📈 Estatísticas do Dataset:")
print(f"  • Total de features extraídas: {len(df_comprehensive.columns) - 3}")
print(f"  • Total de janelas válidas: {len(df_comprehensive)}")
print(f"  • Taxa de sucesso: {len(df_comprehensive) / (len(nsr_records + afib_records) * 2 * 60):.1f}%")

print(f"\n✅ Dataset pronto para visualização e análise!")

---

## 📈 5. Visualização de Features Avançadas

### Comparação: Normal vs. AFib

In [ ]:
# Visualização Abrangente de Features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Análise de Features ECG: Normal vs AFib', fontsize=16, fontweight='bold')

# 1. HRV Tradicional - RMSSD
sns.boxplot(data=df_comprehensive, x='label', y='rmssd', hue='label', ax=axes[0,0], palette='viridis', legend=False)
axes[0,0].set_title('RMSSD (HRV Tradicional)', fontweight='bold')
axes[0,0].set_ylabel('RMSSD (ms)')
axes[0,0].set_xlabel('Tipo de Ritmo')

# 2. Variância Explicada pelo PCA
pca_var_cols = [col for col in df_comprehensive.columns if 'pca_var_ratio_1' in col]
if pca_var_cols:
    sns.boxplot(data=df_comprehensive, x='label', y=pca_var_cols[0], hue='label', ax=axes[0,1], palette='plasma', legend=False)
    axes[0,1].set_title('Variância do 1º Componente PCA', fontweight='bold')
    axes[0,1].set_ylabel('Razão de Variância Explicada')
    axes[0,1].set_xlabel('Tipo de Ritmo')

# 3. Energia ICA
ica_energy_cols = [col for col in df_comprehensive.columns if 'ica_ic1_energy' in col]
if ica_energy_cols:
    sns.boxplot(data=df_comprehensive, x='label', y=ica_energy_cols[0], hue='label', ax=axes[0,2], palette='coolwarm', legend=False)
    axes[0,2].set_title('Energia do 1º Componente ICA', fontweight='bold')
    axes[0,2].set_ylabel('Energia')
    axes[0,2].set_xlabel('Tipo de Ritmo')
    axes[0,2].set_yscale('log')

# 4. Razão LF/HF Espectral
spectral_ratio_cols = [col for col in df_comprehensive.columns if 'spectral_lf_hf_ratio' in col]
if spectral_ratio_cols:
    sns.violinplot(data=df_comprehensive, x='label', y=spectral_ratio_cols[0], hue='label', ax=axes[1,0], palette='Set2', legend=False)
    axes[1,0].set_title('Razão LF/HF (Balanço Autonômico)', fontweight='bold')
    axes[1,0].set_ylabel('Razão LF/HF')
    axes[1,0].set_xlabel('Tipo de Ritmo')

# 5. Scatter PCA vs ICA
pca_mean_cols = [col for col in df_comprehensive.columns if 'pca_pc1_mean' in col]
ica_mean_cols = [col for col in df_comprehensive.columns if 'ica_ic1_mean' in col]
if pca_mean_cols and ica_mean_cols:
    sns.scatterplot(data=df_comprehensive, x=pca_mean_cols[0], y=ica_mean_cols[0], 
                   hue='label', ax=axes[1,1], alpha=0.7, palette='Set1')
    axes[1,1].set_title('Features PCA vs ICA', fontweight='bold')
    axes[1,1].set_xlabel('Média PC1 (PCA)')
    axes[1,1].set_ylabel('Média IC1 (ICA)')
    axes[1,1].legend(title='Tipo de Ritmo')

# 6. Qualidade do Sinal (SNR)
signal_quality_cols = [col for col in df_comprehensive.columns if 'signal_quality_snr' in col]
if signal_quality_cols:
    sns.histplot(data=df_comprehensive, x=signal_quality_cols[0], hue='label', 
                ax=axes[1,2], alpha=0.7, bins=30)
    axes[1,2].set_title('Qualidade do Sinal (SNR)', fontweight='bold')
    axes[1,2].set_xlabel('Razão Sinal-Ruído')
    axes[1,2].set_ylabel('Contagem')

plt.tight_layout()
plt.show()

print("✅ Visualizações geradas com sucesso!")

---

## 📊 6. Análise Estatística: Poder Discriminativo

### Teste de Mann-Whitney U
- **Objetivo**: Identificar features mais discriminativas
- **Método**: Teste não-paramétrico (não assume normalidade)
- **Interpretação**: 
  - `p < 0.05`: Diferença estatisticamente significativa
  - `d` (Cohen's d): Tamanho do efeito (0.2=pequeno, 0.5=médio, 0.8+=grande)

In [ ]:
# Análise Estatística de Features
from scipy import stats

print("📊 ANÁLISE ESTATÍSTICA DE FEATURES")
print("=" * 60)
print("Usando teste Mann-Whitney U para identificar features mais discriminativas")
print("p < 0.05 indica diferença significativa entre Normal e AFib")

# Separar dados Normal e AFib
normal_data = df_comprehensive[df_comprehensive['label'] == 'Normal']
afib_data = df_comprehensive[df_comprehensive['label'] == 'AFib']

# Analisar grupos de features
results = []
for category, features in feature_categories.items():
    print(f"\n{category}:")
    print("-" * (len(category) + 4))
    
    significant_features = 0
    for feature in features:
        if feature in df_comprehensive.columns:
            normal_values = normal_data[feature].dropna()
            afib_values = afib_data[feature].dropna()
            
            if len(normal_values) > 0 and len(afib_values) > 0:
                # Teste Mann-Whitney U (não-paramétrico)
                statistic, p_value = stats.mannwhitneyu(normal_values, afib_values, alternative='two-sided')
                    
                # Tamanho do efeito (Cohen's d)
                pooled_std = np.sqrt((normal_values.var() + afib_values.var()) / 2)
                cohens_d = abs(normal_values.mean() - afib_values.mean()) / pooled_std if pooled_std > 0 else 0
                    
                significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
                    
                print(f"  {feature:<30} p={p_value:.2e} d={cohens_d:.3f} {significance}")
                    
                if p_value < 0.05:
                    significant_features += 1
                    
                results.append({
                    'category': category,
                    'feature': feature,
                    'p_value': p_value,
                    'effect_size': cohens_d,
                    'significant': p_value < 0.05
                })
    
    print(f"  → {significant_features}/{len([f for f in features if f in df_comprehensive.columns])} features significativamente diferentes")

# Resumo dos resultados
df_results = pd.DataFrame(results)
if len(df_results) > 0:
    print(f"\n📈 RESUMO ESTATÍSTICO")
    print("=" * 60)
    
    # Estatísticas por grupo
    group_stats = df_results.groupby('category').agg({
        'significant': ['count', 'sum'],
        'effect_size': ['mean', 'max']
    }).round(3)
    
    print("Desempenho por Categoria:")
    for category in group_stats.index:
        total = group_stats.loc[category, ('significant', 'count')]
        significant = group_stats.loc[category, ('significant', 'sum')]
        mean_effect = group_stats.loc[category, ('effect_size', 'mean')]
        max_effect = group_stats.loc[category, ('effect_size', 'max')]
        
        print(f"  {category:<30}: {significant:2d}/{total:2d} significativas, d médio={mean_effect:.3f}, d máx={max_effect:.3f}")
    
    # Top features
    top_features = df_results.nlargest(10, 'effect_size')
    print(f"\n🏆 Top 10 Features Mais Discriminativas:")
    for i, (_, row) in enumerate(top_features.iterrows(), 1):
        status = "✓" if row['significant'] else "✗"
        print(f"  {i:2d}. {row['feature']:<30} d={row['effect_size']:.3f} {status}")
    
    print(f"\n✅ Total: {df_results['significant'].sum()}/{len(df_results)} features mostram diferenças significativas")
    best_feature = df_results.loc[df_results['effect_size'].idxmax()]
    print(f"🥇 Melhor feature: {best_feature['feature']} (d={best_feature['effect_size']:.3f})")
else:
    print("❌ Nenhum resultado para analisar.")

---

## 🚀 7. Classificação com XGBoost

### Por que XGBoost?

✅ **Vantagens para Dados Médicos**:
- Lida bem com features mistas (HRV + morfológicas)
- Regularização integrada (previne overfitting)
- Interpretabilidade via importância de features
- Excelente desempenho em dados tabulares

### Features Selecionadas
- **RMSSD**: Medida tradicional de HRV
- **Variância PCA**: Captura variações morfológicas
- **Energia ICA**: Possível assinatura de ondas fibrilatórias
- **Outras features discriminativas**

In [ ]:
# Preparar dados para XGBoost
print("🚀 PREPARANDO DADOS PARA CLASSIFICAÇÃO XGBOOST")
print("=" * 60)

print(f"Dimensões do dataset: {df_comprehensive.shape}")

# Features-chave para classificação
key_features = []

# 1. RMSSD (HRV)
if 'rmssd' in df_comprehensive.columns:
    key_features.append('rmssd')
    print(f"✓ Feature RMSSD encontrada")

# 2. Variância do 1º Componente PCA
pca_var_cols = [col for col in df_comprehensive.columns if 'pca_var_ratio_1' in col]
if pca_var_cols:
    key_features.append(pca_var_cols[0])
    print(f"✓ Variância PCA encontrada: {pca_var_cols[0]}")

# Features discriminativas adicionais
additional_features = []
for feature in ['sdnn', 'mean_rr', 'pca_pc1_mean', 'spectral_lf_hf_ratio', 'ica_ic3_energy']:
    if feature in df_comprehensive.columns:
        additional_features.append(feature)

print(f"\n📊 Features primárias: {key_features}")
print(f"📊 Features adicionais: {additional_features}")

# Combinar features
all_features = key_features + additional_features
print(f"\n✅ Total de features para classificação: {len(all_features)}")
print(f"📋 Lista de features: {all_features}")

In [ ]:
# Implementação da Classificação XGBoost
print("🤖 CLASSIFICAÇÃO DE ANOMALIAS ECG COM XGBOOST")
print("=" * 60)

# Preparar dataset
X = df_comprehensive[all_features].copy()
y = df_comprehensive['label'].copy()

# Codificar labels (XGBoost precisa de labels numéricos)
label_mapping = {'Normal': 0, 'AFib': 1}
y_encoded = y.map(label_mapping)

print(f"Dimensões: {X.shape}")
print(f"Distribuição de classes:")
print(y.value_counts())

# Tratar valores faltantes
missing_count = X.isnull().sum().sum()
if missing_count > 0:
    print(f"\n⚠️ Tratando {missing_count} valores faltantes...")
    X = X.fillna(X.median())
else:
    print("\n✅ Nenhum valor faltante detectado!")

# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\n📊 Conjunto de treino: {X_train.shape[0]} amostras")
print(f"📊 Conjunto de teste: {X_test.shape[0]} amostras")
print(f"   Treino - Normal: {np.sum(y_train==0)}, AFib: {np.sum(y_train==1)}")
print(f"   Teste  - Normal: {np.sum(y_test==0)}, AFib: {np.sum(y_test==1)}")

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Inicializar XGBoost com parâmetros otimizados para dados médicos
xgb_classifier = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    reg_alpha=0.1,
    reg_lambda=0.1
)

print(f"\n✅ Modelo XGBoost inicializado com parâmetros otimizados para dados médicos")

---

## 📊 8. Treinamento e Avaliação do Modelo

In [ ]:
# Treinar e Avaliar Modelo
print("🎓 TREINANDO MODELO XGBOOST")
print("=" * 60)

# Treinar modelo
print("Treinando classificador XGBoost...")
xgb_classifier.fit(X_train_scaled, y_train)
print("✅ Treinamento concluído!")

# Fazer predições
y_pred = xgb_classifier.predict(X_test_scaled)
y_pred_proba = xgb_classifier.predict_proba(X_test_scaled)[:, 1]

# Calcular métricas
accuracy = (y_pred == y_test).mean()
auc_score = roc_auc_score(y_test, y_pred_proba)

print(f"\n📊 MÉTRICAS DE DESEMPENHO DO MODELO")
print("=" * 40)
print(f"🎯 Acurácia: {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"📈 AUC Score: {auc_score:.3f}")

# Relatório de classificação detalhado
print(f"\n📋 Relatório de Classificação Detalhado:")
target_names = ['Normal', 'AFib']
print(classification_report(y_test, y_pred, target_names=target_names))

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print(f"\n🔢 Matriz de Confusão:")
print(f"                Predito")
print(f"                Normal  AFib")
print(f"Real Normal     {cm[0,0]:6d} {cm[0,1]:5d}")
print(f"     AFib       {cm[1,0]:6d} {cm[1,1]:5d}")

# Validação cruzada
print(f"\n🔄 RESULTADOS DA VALIDAÇÃO CRUZADA")
print("=" * 40)
cv_scores = cross_val_score(xgb_classifier, X_train_scaled, y_train, cv=5, scoring='roc_auc')
print(f"5-Fold CV AUC: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
print(f"Folds individuais: {[f'{score:.3f}' for score in cv_scores]}")

print(f"\n✅ Classificação XGBoost concluída com sucesso!")

---

## 📈 9. Análise de Importância de Features

### Interpretação do Modelo

In [ ]:
# Análise de Importância de Features
print("🔍 ANÁLISE DE IMPORTÂNCIA DE FEATURES")
print("=" * 60)

# Obter importância do XGBoost
feature_importance = xgb_classifier.feature_importances_
feature_names = all_features

# Criar dataframe de importância
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False).reset_index(drop=True)

print("🏆 Top 10 Features Mais Importantes:")
print("-" * 40)
for i, (_, row) in enumerate(importance_df.head(10).iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")

# Visualizações
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Resultados da Classificação XGBoost - ECG', fontsize=16, fontweight='bold')

# 1. Importância de Features
top_features = importance_df.head(10)
sns.barplot(data=top_features, y='feature', x='importance', ax=axes[0,0], palette='viridis')
axes[0,0].set_title('Top 10 Features Mais Importantes', fontweight='bold')
axes[0,0].set_xlabel('Score de Importância')

# 2. Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
axes[0,1].plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {auc_score:.3f})')
axes[0,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Classificador Aleatório')
axes[0,1].set_xlim([0.0, 1.0])
axes[0,1].set_ylim([0.0, 1.05])
axes[0,1].set_xlabel('Taxa de Falsos Positivos')
axes[0,1].set_ylabel('Taxa de Verdadeiros Positivos')
axes[0,1].set_title('Curva ROC', fontweight='bold')
axes[0,1].legend(loc="lower right")
axes[0,1].grid(True, alpha=0.3)

# 3. Heatmap da Matriz de Confusão
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1,0],
            xticklabels=['Normal', 'AFib'], yticklabels=['Normal', 'AFib'])
axes[1,0].set_title('Matriz de Confusão', fontweight='bold')
axes[1,0].set_xlabel('Label Predito')
axes[1,0].set_ylabel('Label Verdadeiro')

# 4. Distribuição de Probabilidades de Predição
prob_df = pd.DataFrame({
    'probability': y_pred_proba,
    'true_label': ['Normal' if x == 0 else 'AFib' for x in y_test]
})
sns.histplot(data=prob_df, x='probability', hue='true_label', ax=axes[1,1], alpha=0.7, bins=20)
axes[1,1].set_title('Distribuição de Probabilidades de Predição', fontweight='bold')
axes[1,1].set_xlabel('Probabilidade AFib')
axes[1,1].set_ylabel('Contagem')
axes[1,1].axvline(x=0.5, color='red', linestyle='--', alpha=0.7, label='Limiar de Decisão')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Análise de features-chave
print(f"\n🔑 ANÁLISE DE FEATURES-CHAVE")
print("=" * 40)
rmssd_importance = importance_df[importance_df['feature'] == 'rmssd']['importance'].values
pca_var_importance = importance_df[importance_df['feature'].str.contains('pca_var_ratio_1', na=False)]['importance'].values

if len(rmssd_importance) > 0:
    print(f"RMSSD importância: {rmssd_importance[0]:.4f} (Rank: {importance_df[importance_df['feature'] == 'rmssd'].index[0] + 1})")
if len(pca_var_importance) > 0:
    pca_feature_name = importance_df[importance_df['feature'].str.contains('pca_var_ratio_1', na=False)]['feature'].iloc[0]
    print(f"Variância PCA importância: {pca_var_importance[0]:.4f} (Rank: {importance_df[importance_df['feature'] == pca_feature_name].index[0] + 1})")

print(f"\n✅ Modelo classifica anomalias ECG com {accuracy:.1%} de acurácia!")
print(f"✅ Poder discriminativo forte (AUC: {auc_score:.3f})")

---

## 💡 10. Insights Clínicos

### Interpretação dos Resultados

In [ ]:
# Interpretação do Modelo e Insights Clínicos
print("🔬 INTERPRETAÇÃO DO MODELO & INSIGHTS CLÍNICOS")
print("=" * 60)

# Analisar desempenho específico de RMSSD
if 'rmssd' in all_features:
    normal_rmssd = df_comprehensive[df_comprehensive['label'] == 'Normal']['rmssd']
    afib_rmssd = df_comprehensive[df_comprehensive['label'] == 'AFib']['rmssd']
    
    print(f"📊 Análise RMSSD:")
    print(f"  Ritmo Normal - Média: {normal_rmssd.mean():.2f} ms, Desvio: {normal_rmssd.std():.2f} ms")
    print(f"  Ritmo AFib   - Média: {afib_rmssd.mean():.2f} ms, Desvio: {afib_rmssd.std():.2f} ms")
    print(f"  Diferença: {abs(afib_rmssd.mean() - normal_rmssd.mean()):.2f} ms")
    print(f"  💡 Insight clínico: {'AFib mostra maior variabilidade' if afib_rmssd.mean() > normal_rmssd.mean() else 'Normal mostra maior variabilidade'}")

# Encontrar feature PCA
pca_var_cols = [col for col in all_features if 'pca_var_ratio_1' in col]
if pca_var_cols:
    pca_feature = pca_var_cols[0]
    normal_pca = df_comprehensive[df_comprehensive['label'] == 'Normal'][pca_feature]
    afib_pca = df_comprehensive[df_comprehensive['label'] == 'AFib'][pca_feature]
    
    print(f"\n📊 Análise Variância 1º Componente PCA:")
    print(f"  Ritmo Normal - Média: {normal_pca.mean():.4f}, Desvio: {normal_pca.std():.4f}")
    print(f"  Ritmo AFib   - Média: {afib_pca.mean():.4f}, Desvio: {afib_pca.std():.4f}")
    print(f"  Diferença: {abs(afib_pca.mean() - normal_pca.mean()):.4f}")
    print(f"  💡 Insight clínico: {'AFib mostra mais variação morfológica' if afib_pca.mean() > normal_pca.mean() else 'Normal mostra mais variação morfológica'}")

# Análise de confiança das predições
high_confidence_correct = np.sum((y_pred_proba > 0.8) & (y_pred == y_test)) + np.sum((y_pred_proba < 0.2) & (y_pred == y_test))
high_confidence_total = np.sum((y_pred_proba > 0.8) | (y_pred_proba < 0.2))
medium_confidence_total = np.sum((y_pred_proba >= 0.4) & (y_pred_proba <= 0.6))

print(f"\n📈 ANÁLISE DE CONFIANÇA DAS PREDIÇÕES")
print("=" * 40)
print(f"Predições de alta confiança (>80% ou <20%): {high_confidence_total}/{len(y_test)} ({high_confidence_total/len(y_test)*100:.1f}%)")
print(f"Acurácia em alta confiança: {high_confidence_correct/high_confidence_total*100 if high_confidence_total > 0 else 0:.1f}%")
print(f"Predições incertas (40-60%): {medium_confidence_total}/{len(y_test)} ({medium_confidence_total/len(y_test)*100:.1f}%)")

# Resumo para suporte à decisão clínica
print(f"\n🏥 RESUMO PARA SUPORTE À DECISÃO CLÍNICA")
print("=" * 50)
quality = 'Excelente' if accuracy > 0.9 else 'Bom' if accuracy > 0.8 else 'Moderado'
auc_quality = 'Excelente' if auc_score > 0.9 else 'Bom' if auc_score > 0.8 else 'Moderado'
print(f"✅ Acurácia do Modelo: {accuracy:.1%} - {quality}")
print(f"✅ AUC Score: {auc_score:.3f} - Poder discriminativo {auc_quality}")
print(f"✅ Utilidade Clínica: Pode auxiliar em triagem automatizada de AFib a partir de dados ECG")

# Prontidão para deploy
print(f"\n🚀 PRONTIDÃO PARA IMPLANTAÇÃO")
print("=" * 40)
print(f"✅ Dataset balanceado: Normal ({np.sum(y_encoded == 0)}) vs AFib ({np.sum(y_encoded == 1)}) amostras")
print(f"✅ Validação cruzada: {cv_scores.mean():.3f} ± {cv_scores.std():.3f} (desempenho consistente)")
print(f"✅ Estabilidade de features: {len(all_features)} features robustas extraídas")

---

## 🎯 11. Conclusões Principais

### 🏆 Resultados Alcançados

#### Desempenho do Modelo
- ✅ **Acurácia elevada**: Classificação precisa entre Normal e AFib
- ✅ **AUC excelente**: Forte poder discriminativo
- ✅ **Validação cruzada consistente**: Modelo robusto e generaliza bem

#### Features Mais Importantes

**1. 🥇 ICA IC3 Energy - A Estrela do Projeto**
- **Cohen's d astronômico**: Separação quase perfeita entre classes
- **Explicação provável**: O 3º componente independente isolou as **ondas fibrilatórias** ('ondas f')
- **Analogia**: Como um engenheiro de som que consegue isolar o ruído de fundo em uma gravação

**2. 🥈 Variâncias PCA (Componentes 3, 4 e 5)**
- Capturam variações morfológicas significativas
- AFib apresenta maior variabilidade na forma dos batimentos

**3. 🥉 Features HRV Tradicionais**
- RMSSD, SDNN, Mean RR: Todas significativas
- Confirmam a hipótese: AFib tem maior irregularidade

#### Por que ICA IC3 Energy Funcionou Tão Bem?

**Analogia Musical 🎶**

**Ritmo Normal**: Banda tocando limpo
- 🥁 Bateria (QRS) - forte e dominante
- 🎸 Baixo (onda T) - rítmico

**Ritmo AFib**: Mesma banda + pandeiro caótico ao fundo
- 🥁 Bateria (QRS)
- 🎸 Baixo (onda T)  
- 🪘 **Pandeiro caótico (ondas fibrilatórias)** ← ICA isolou isto!

**ICA separou as "faixas de áudio":**
- IC1: Provavelmente isolou o QRS (bateria)
- IC2: Talvez a onda T (baixo)
- **IC3: ISOLOU O PANDEIRO CAÓTICO** (ondas fibrilatórias)

A energia do IC3 mede o "volume" desse pandeiro:
- **Normal**: Quase silêncio (energia baixa)
- **AFib**: Barulho constante (energia alta)

---

## ⚠️ 12. Limitações e Trabalhos Futuros

### Limitações Identificadas

#### 1. Tamanho da Amostra
- **Atual**: 10 pacientes (5 Normal + 5 AFib)
- **Recomendação**: Utilizar todos os registros disponíveis
  - nsrdb: 18 pacientes
  - afdb: 23 pacientes
- **Benefício**: Maior diversidade e melhor generalização

#### 2. Janelas de 30 Segundos
- ✅ **Vantagens**: Computacionalmente eficiente, adequado para HRV e morfologia
- ⚠️ **Limitações**: Insuficiente para análise espectral completa
  - Não resolve bem a banda VLF (< 0.04 Hz)
  - LF/HF ratio não é confiável
- **Recomendação**: Janelas de 5 minutos para análise espectral robusta

#### 3. Validação Cruzada
- **Atenção**: Risco de data leakage
- **Problema**: Múltiplas janelas do mesmo paciente em treino e teste
- **Solução**: Implementar validação cruzada a nível de paciente

### Trabalhos Futuros

1. **Expandir Dataset**: Incluir todos os registros disponíveis
2. **Validação por Paciente**: Implementar Leave-One-Patient-Out CV
3. **Análise de Janelas Maiores**: Testar com 5 minutos para análise espectral
4. **Explorar IC3 Detalhadamente**: Visualizar e entender o que foi isolado
5. **Testar Outros Classificadores**: Random Forest, SVM, Redes Neurais
6. **Detecção em Tempo Real**: Adaptar para streaming de ECG

---

## 📚 13. Referências

### Bases de Dados
- **MIT-BIH Normal Sinus Rhythm Database**: Goldberger et al., PhysioNet
- **MIT-BIH Atrial Fibrillation Database**: Moody & Mark, PhysioNet

### Métodos
- **Análise HRV**: Task Force Guidelines (1996)
- **PCA em ECG**: Transformações lineares para análise morfológica
- **ICA em ECG**: Separação cega de fontes para remoção de artefatos
- **XGBoost**: Chen & Guestrin (2016)

### Ferramentas
- **WFDB Python**: Biblioteca para processamento de sinais fisiológicos
- **scikit-learn**: Ferramentas de machine learning
- **XGBoost**: Gradient boosting otimizado

---

## 🙏 Agradecimentos

Obrigado pela atenção!

**Contato**: [Seu email/informações]  
**Código**: [Link do GitHub]

---

### 💻 Ambiente Técnico
- Python 3.x
- Principais bibliotecas: wfdb, scikit-learn, xgboost, pandas, numpy, matplotlib, seaborn
- Processamento de sinais: scipy

In [ ]:
# Célula final para gerar um resumo executivo
print("=" * 70)
print(" " * 15 + "RESUMO EXECUTIVO DO PROJETO")
print("=" * 70)
print(f"\n📊 DADOS")
print(f"   • Pacientes: 10 (5 Normal + 5 AFib)")
print(f"   • Amostras totais: {len(df_comprehensive)}")
print(f"   • Features extraídas: {len(all_features)}")
print(f"\n🎯 MODELO")
print(f"   • Algoritmo: XGBoost")
print(f"   • Acurácia: {accuracy:.1%}")
print(f"   • AUC Score: {auc_score:.3f}")
print(f"   • Validação Cruzada: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
print(f"\n🏆 TOP 3 FEATURES")
for i, (_, row) in enumerate(importance_df.head(3).iterrows(), 1):
    print(f"   {i}. {row['feature']}: {row['importance']:.4f}")
print(f"\n✅ CONCLUSÃO")
print(f"   O modelo demonstra excelente capacidade de distinguir entre")
print(f"   ritmo sinusal normal e fibrilação atrial usando features")
print(f"   avançadas de processamento de sinais ECG.")
print("=" * 70)